In [32]:
from torchvision import transforms
from torchvision.datasets import FashionMNIST
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [34]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = FashionMNIST(root='./data', train=True, transform=transform, download=True)
testset = FashionMNIST(root='./data', train=False, transform=transform, download=True)